No costs!  Bases and jobs have simple sum requirement.  Mods use decision trees to define eligibility.

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates
using MAT
using Serialization

struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
    cost::Int64
end

const ability_none = 999
const ability_imitative = 10
const ability_reciprocating = 20
const ability_romantic = 30
const ability_bibliophile = 40
const ability_melodious = 50
const ability_iconoclast = 90
const ability_intoxicating = 100
const ability_otaku = 110
const ability_sincere = 120
const ability_cowardly = 130


ability_name = Dict(
    ability_imitative => "Imitative*",
    ability_reciprocating => "Reciprocating*",
    ability_romantic => "Romantic*",
    ability_bibliophile => "Bibliophile*",
    ability_melodious => "Melodious*",
    ability_iconoclast => "Iconoclast*",
    ability_intoxicating => "Intoxicating*",
    ability_otaku => "Otaku*",
    ability_sincere => "Sincere*",
    ability_cowardly => "Cowardly*",
)

function fx_imitative(a::Spud, b::Spud)
    if b.h > a.h
        a = Spud(a.name, b.h, a.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_reciprocating(a::Spud, b::Spud)
    if b.f > a.f
        a = Spud(a.name, a.h, b.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_romantic(a::Spud, b::Spud)
    if b.p > a.l
        a = Spud(a.name, a.h, a.f, b.p, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_bibliophile(a::Spud, b::Spud)
    if b.r > a.l
        a = Spud(a.name, a.h, a.f, b.r, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_melodious(a::Spud, b::Spud)
    if b.p > b.r && b.p > b.s
        a = Spud(a.name, a.h, a.f, a.r, a.p + 3, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_iconoclast(a::Spud, b::Spud)
    v = sum([b.h,b.f,b.l,b.p,b.r,b.s] .== MXV)
    if v >= 3
        aa = [a.h, a.f, a.l, a.p, a.r, a.s]
        aa2 = map(x -> div(x, 4), aa)
        r = aa .+ aa2
        a = Spud(a.name, r[1], r[2], r[3], r[4], r[5], r[6], a.a1)
    end
    return [a, b]
end

function fx_intoxicating(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.f, b.a1)
    return [a, b]
end

function fx_otaku(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.h, b.s, b.a1)
    return [a, b]
end


function fx_sincere(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f + 2, b.f + 2, b.p, b.r, b.s, b.a1)
    return [a, b]
end

function fx_cowardly(a::Spud, b::Spud)
    if (a.p < b.p) || (a.r < b.r) || (a.s < b.s)
        a = Spud(a.name, a.h + 2, a.f - 2, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

ability_fx = Dict(
    ability_imitative => fx_imitative,
    ability_reciprocating => fx_reciprocating,
    ability_romantic => fx_romantic,
    ability_bibliophile => fx_bibliophile,
    ability_melodious => fx_melodious,
    ability_iconoclast => fx_iconoclast,
    ability_intoxicating => fx_intoxicating,
    ability_otaku => fx_otaku,
    ability_sincere => fx_sincere,
    ability_cowardly => fx_cowardly,
)

const MXS = 100
const MXV = 9
const MNV = 1

# function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
#     #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
#     return h*(f+l) + f*(p+r+s)
# end

# function cost(a::Spud)::Int64
#     if a.a1 == ability_none
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s)
#     else
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s) + ability_cost[a.a1](a)
#     end
# end

function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end

function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

eval_battle (generic function with 1 method)

In [2]:

function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    cs = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
        cs[ii] = as[ii].cost
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s, cost = cs)
    return df
end



function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none, -1)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1], df[i, :cost])
    end
    return as
end



struct NashEquilibrium2p
    w1::Array{Float64}
    w2::Array{Float64}
    v::Float64
end


function payoffs2(set1::Array{Spud}, set2::Array{Spud})::Array{Int64}
    n1 = length(set1)
    n2 = length(set2)
    payoffs = Array{Int64}(undef, (n1, n2))
    for i in 1:n1
        for j in 1:n2
            payoffs[i,j] = eval_battle(set1[i], set2[j])
        end
    end
    return payoffs
end

function ffp2(set1::Array{Spud}, set2::Array{Spud}, nits::Int64)::NashEquilibrium2p
    n1 = length(set1)
    n2 = length(set2)
    payoffs = payoffs2(set1, set2)
    v1 = [1 for i in 1:n1]
    v2 = [1 for i in 1:n2]
    s1 = [sum(payoffs[i, :] .* v2) for i in 1:n1]
    s2 = [sum(payoffs[:, i] .* v1) for i in 1:n2]
    for it in 1:nits
        cands1 = findall(s1 .== maximum(s1))
        i1 = rand(cands1)
        cands2 = findall(s2 .== minimum(s2))
        i2 = rand(cands2)
        v1[i1] += 1
        s2 = s2 .+ payoffs[i1, :]
        v2[i2] += 1
        s1 = s1 .+ payoffs[:, i2]
    end
    w1 = v1./sum(v1)
    w2 = v2./sum(v2)
    val = sum([payoffs[i, j] * w1[i] * w2[j] for i in 1:n1 for j in 1:n2])
    return NashEquilibrium2p(w1, w2, val)
end

function ffp(nash_env, nits)
    n_nash = length(nash_env)
    i_lose = Array{Int}(undef, (n_nash, n_nash))
    n_lose = Array{Int}(undef, n_nash)
    for i in 1:n_nash
        n_lose[i] = 0
        ff = nash_env[i]
        for j in 1:n_nash
            if eval_battle(ff, nash_env[j]) ==-1
                n_lose[i] += 1
                i_lose[i, n_lose[i]] = j
            end
        end
    end
    counts = [0 for i in 1:n_nash]
    wins = [0 for i in 1:n_nash]
    for i in 1:n_nash
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    for iter in 1:nits
        wc = 2 .* wins .+ counts
        ind_winners = findall(wc .== maximum(wc))
        i = rand(ind_winners)
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    return counts
end

mutable struct BattleView
    round::Int64
    own::Array{Spud}
    owncount::Array{Int64}
    deployed::Array{Bool}
    obs::Array{Spud}
    obscount::Array{Int64}
end

mutable struct AiPlayer
    genteam::Function
    pick::Function
    create::Function
end

n_copy = 3

function initialize_game(team1::Array{Spud}, team2::Array{Spud})::Array{BattleView}
    teams = [team1, team2]
    bvs = Array{BattleView}(undef, 0)
    for team in teams
        bv = BattleView(0, team, [n_copy for i in 1:length(team)], [false for i in 1:length(team)], [], [])
        append!(bvs, [bv])
    end
    return bvs
end

function observe_fighters(game::Array{BattleView}, choices::Array{Int64})::Array{BattleView}
    for i in 1:2
        j = 3- i
        if choices[i] != 0
            game[i].deployed[choices[i]] = true
            ff = game[i].own[choices[i]]
            if ff in game[j].obs
                0
            else
                game[j].obs = append!(game[j].obs, [ff])
                game[j].obscount = append!(game[j].obscount, [game[i].owncount[choices[i]]])
            end
        end
    end
    return game
end

function jobify_fighter_counts(game::Array{BattleView}, choices::Array{Int64}, job::Array{Int64})::Array{BattleView}
    for i in 1:2
        j = 3- i
        if choices[i] != 0
            game[i].owncount[choices[i]] = game[i].owncount[choices[i]] + job[i]
            ff = game[i].own[choices[i]]
            if ff in game[j].obs
                ind = findall(x -> x==ff, game[j].obs)[1]
                game[j].obscount[ind] = game[i].owncount[choices[i]]
            else
                0
            end
        end
    end
    return game
end

function play_combat(game::Array{BattleView}, choices::Array{Int64}, verbose::Bool = false)::Array{BattleView}
    round = game[1].round + 1
    fighters = [game[i].own[choices[i]] for i in 1:2]
    for i in 1:2
        @assert game[i].owncount[choices[i]] >= 0
    end
    if verbose
        print("Round ")
        print(round)
        print(": #A")
        print(choices[1])
        print(" ")
        print(fighters[1])
        print(" vs #B")
        print(choices[2])
        print(" ")
        println(fighters[2])
    end
    outcome = eval_battle(fighters[1], fighters[2])
    if verbose
        if outcome == 0
            println("Tied!")
        else
            if outcome == 1
                print(fighters[1])
            end
            if outcome == -1
                print(fighters[2])
            end
            println(" wins!")
        end
    end
    game = observe_fighters(game, choices)
    if outcome > -1
        game = jobify_fighter_counts(game, choices, [0, -1])
    end
    if outcome < 1
        game = jobify_fighter_counts(game, choices, [-1, 0])
    end
    for i in 1:2
        game[i].round = round
    end
    return game
end

function play_add_spud(game::Array{BattleView}, player::Int64, ff::Spud, verbose::Bool = false)::Array{BattleView}
    game[player].own = append!(game[player].own, [ff])
    game[player].owncount = append!(game[player].owncount, [n_copy])
    game[player].deployed = append!(game[player].deployed, [false])
    if verbose
        print("Player ")
        print(player)
        print(" adds #")
        print(["A", "B"][player])
        print(length(game[player].own))
        print(" ")
        println(ff)
    end
    return game
end

function play_is_game_over(game::Array{BattleView})::Bool
    for bv in game
        if sum(bv.owncount) == 0
            return true
        end
    end
    return false
end

function generate_rand_team(env, counts, nteam)
    cc = cumsum(counts)
    team = Array{Spud}(undef, nteam)
    for i in 1:nteam
        tmp = rand(1:sum(counts))
        ind = sum(cc .< tmp)+1
        team[i] = env[ind]
    end
    return team
end

function ai0_pick(a::BattleView)::Int64
    inds = findall(a.owncount .> 0)
    return rand(inds)
end

function ai0_create(a::BattleView)::Spud
    return generate_rand_team(nash_env, counts, 1)[1]
end

function ai1_create(a::BattleView)::Spud
    ops = a.obs[a.obscount .> 0]
    ebs = [eval_battle_list(ff, ops) for ff in nash_env]
    cands = nash_env[ebs .== maximum(ebs)]
    return rand(cands)
end

function ai2_create(a::BattleView)::Spud
    ops = a.obs[a.obscount .> 0]
    ebs = [eval_battle_list(ff, ops) for ff in nash_env]
    if maximum(ebs) == length(ops)
        cands = nash_env[ebs .== maximum(ebs)]
        return rand(cands)
    else
        ebs2 = [eval_battle_list(ff, ops) for ff in lib]
        if maximum(ebs2) > maximum(ebs)
            cands = lib[ebs2 .== maximum(ebs2)]
            return rand_rename(rand(cands))
        else
            cands = nash_env[ebs .== maximum(ebs)]
            return rand(cands)
        end
    end
end


# picks Spud most likely to win
function ai_greedy_pick(a::BattleView)::Int64
    if length(a.obs[a.obscount .> 0]) == 0
        return rand(findall(a.owncount .> 0))
    end
    res = ffp2(a.own[a.owncount .> 0], a.obs[a.obscount .> 0], 100)
    ind = findall(a.owncount .> 0)[rand(findall(res.w1 .== maximum(res.w1)))]
    return ind
end

# picks from spuds already deployed
function ai_conservative_pick(a::BattleView)::Int64
    own = a.own[a.owncount .> 0 .&& a.deployed]
    owninds = findall(a.owncount .> 0 .&& a.deployed)
    if length(own) == 0
        # pick greedy
        return ai_greedy_pick(a)
    end
    ops = a.obs[a.obscount .> 0]
    res = ffp2(own, ops, 100)
    ind = owninds[rand(findall(res.w1 .== maximum(res.w1)))]
    return ind
end

# picks Spud based on 1-round nash
function ai_myopic_pick(a::BattleView)::Int64
    if length(a.obs[a.obscount .> 0]) == 0
        return rand(findall(a.owncount .> 0))
    end
    res = ffp2(a.own[a.owncount .> 0], a.obs[a.obscount .> 0], 100)
    cc = cumsum(res.w1)
    ind = sum(cc .< rand()) + 1
    return findall(a.owncount .> 0)[ind]
end

creation_rounds = [5, 10, 15]

function play_game(players::Array{AiPlayer}, verbose::Bool = true)::Int64
    ai1 = players[1]
    ai2 = players[2]
    ais = players
    team1 = ai1.genteam()
    team2 = ai2.genteam()
    game = initialize_game(team1, team2)
    while !play_is_game_over(game)
        # play a round
        choice1 = ai1.pick(game[1])
        choice2 = ai2.pick(game[2])
        game = play_combat(game, [choice1, choice2], verbose)
        if verbose
            for i in 1:2
                println(game[i].owncount)
            end
        end
        if game[1].round in creation_rounds
            for i in 1:2
                ff = ais[i].create(game[i])
                game = play_add_spud(game, i, ff, verbose)
            end
        end
    end
    if sum(game[1].owncount) == 0
        if sum(game[2].owncount) == 0
            return 0
        else
            return -1
        end
    end
    return 1
end

# ai1 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai_greedy_pick, ai1_create)
# ai2 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai_greedy_pick, ai1_create)
# ais = [ai1, ai2]
# play_game(ais, true)

#@time lib = sample_library(1.01, [999]);

function filter_nondominated(as::Array{Spud})::Array{Spud}
    df = spuds_to_df(as)
    mat = Array{Int64}(undef, (length(as), 6))
    mat[:, 1] = df.h
    mat[:, 2] = df.f
    mat[:, 3] = df.l
    mat[:, 4] = df.p
    mat[:, 5] = df.r
    mat[:, 6] = df.s;
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:6
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

function spud_to_vec(a::Spud)::Array{Int64}
    return [a.h, a.f, a.l, a.p, a.r, a.s]
end

struct SpudBase
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
    cost::Int64
end

function spud_to_vec(a::SpudBase)::Array{Int64}
    return [a.h, a.f, a.l, a.p, a.r, a.s]
end
                                                                                    
struct SpudMod
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
    minh::Int64
    minf::Int64
    minl::Int64
    minp::Int64
    minr::Int64
    mins::Int64
    cost::Int64
end
                                                                                        
function spuds_to_df(as::Array{SpudBase})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    cs = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
        cs[ii] = as[ii].cost
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s, cost = cs)
    return df
end
                                                                     

spuds_to_df (generic function with 2 methods)

In [3]:
const discount = 20

function rename_base(a::SpudBase, s::String)::SpudBase
    return SpudBase(s, a.h,a.f,a.l,a.p,a.r,a.s,a.a1,a.cost)
end

function rename_job(a::SpudMod, s::String)::SpudMod
    return SpudMod(s, a.h,a.f,a.l,a.p,a.r,a.s,a.a1,a.minh,a.minf,a.minl,a.minp,a.minr,a.mins,a.cost)
end

function check_validity(a::Spud)::Bool
    return minimum([a.h,a.f,a.l,a.p,a.r,a.s]) >= MNV
end

function combine_base_job(b::SpudBase, j::SpudMod)::Spud
    v = [b.h,b.f,b.l,b.p,b.r,b.s]
    c = b.cost
    a1 = b.a1
    if a1 == 999
        a1 = j.a1
    end
    w = [j.h,j.f,j.l,j.p,j.r,j.s]
    m = [j.minh,j.minf,j.minl,j.minp,j.minr,j.mins]
    c2 = j.cost
    if sum(v .< m) == 0
        c2 -= discount
    end
    z = v .+ w
    z[z .> MXV] .= MXV
    return Spud(string(j.name, " ", b.name),z[1],z[2],z[3],z[4],z[5],z[6],a1,c+c2)
end


function combine_spud_mod(b::Spud, j::SpudMod)::Spud
    v = [b.h,b.f,b.l,b.p,b.r,b.s]
    c = b.cost
    a1 = b.a1
    if a1 == 999
        a1 = j.a1
    end
    w = [j.h,j.f,j.l,j.p,j.r,j.s]
    m = [j.minh,j.minf,j.minl,j.minp,j.minr,j.mins]
    c2 = j.cost
    # require spud to meet minimums
    if sum(v .< m) > 0 || minimum(v .+ w) < MNV
        return b
    end
    z = v .+ w
    z[z .> MXV] .= MXV
    return Spud(string(j.name, " ", b.name),z[1],z[2],z[3],z[4],z[5],z[6],a1,c+c2)
end

combine_spud_mod (generic function with 1 method)

In [4]:
function get_spuds_with_value(scores::Array{Int64}, vmin::Int64, vmax::Int64)::Array{Spud}
    inds = findall(scores .>= vmin .&& scores .<= vmax)
    return map(x -> Spud("spud",x[1],x[2],x[3],x[4],x[5],x[6],ability_none,scores[x]), inds)
end

function choice_wo(x::Array, n::Int64)::Array
    w = [rand() for i in 1:length(x)]
    return x[sortperm(w)][1:n]
end

choice_wo (generic function with 1 method)

#### Load pre-computed scores (from game_designL)

In [5]:
scores = matread("temp_desE_scores400.mat")["scores"];
global_ne = get_spuds_with_value(scores, 95, 100);

In [6]:
wt = [1,1,2,1,1,1]
#wt = [13,14,19,10,10,10]
sums = [sum(spud_to_vec(s) .* wt) for s in global_ne];
minimum(sums), maximum(sums), minimum(sums)/maximum(sums)

(26, 45, 0.5777777777777777)

In [7]:
spuds_to_df(global_ne[sortperm(sums)[1:8]])

Row,name,h,f,l,p,r,s,a1,cost
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,spud,1,7,1,1,8,7,999,96
2,spud,1,6,1,8,1,8,999,95
3,spud,1,7,1,1,7,8,999,97
4,spud,1,6,1,1,8,8,999,97
5,spud,1,6,1,1,7,9,999,96
6,spud,1,9,1,8,6,1,999,100
7,spud,1,9,1,7,7,1,999,99
8,spud,1,6,1,9,8,1,999,100


In [8]:
spuds_to_df(global_ne[sortperm(-sums)[1:5]])

Row,name,h,f,l,p,r,s,a1,cost
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,spud,9,1,4,9,9,9,999,97
2,spud,4,9,9,2,9,2,999,100
3,spud,3,9,9,2,9,3,999,100
4,spud,9,1,4,9,9,8,999,96
5,spud,9,1,4,9,8,9,999,97


In [9]:
# get all vectors with sum 293
temp = findall(scores .>= -999);
sums = [t[1]*wt[1]+t[2]*wt[2]+t[3]*wt[3]+t[4]*wt[4]+t[5]*wt[5]+t[6]*wt[6] for t in temp];
temp = temp[sums .<= 24 .&& sums .>= 24];
bases = map((x, i) -> Spud(string("b", i),x[1],x[2],x[3],x[4],x[5],x[6],ability_none,0), temp, 1:length(temp));
length(bases)

10862

In [10]:
base_vs = copy(transpose(reduce(hcat, map(spud_to_vec, bases))));

In [11]:
maximum([scores[CartesianIndex((base_vs[i, :]...))] for i in 1:size(base_vs)[1]])

87

In [12]:
spuds_to_df(bases[sortperm(-[scores[CartesianIndex((base_vs[i, :]...))] for i in 1:size(base_vs)[1]])][1:10])

Row,name,h,f,l,p,r,s,a1,cost
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,b21,8,1,6,1,1,1,999,0
2,b29,6,1,7,1,1,1,999,0
3,b2146,7,1,6,1,1,2,999,0
4,b13,9,2,5,1,1,1,999,0
5,b22,7,2,6,1,1,1,999,0
6,b57,9,1,5,2,1,1,999,0
7,b66,7,1,6,2,1,1,999,0
8,b347,9,1,5,1,2,1,999,0
9,b356,7,1,6,1,2,1,999,0


#### analyze restrictions for mods

In [13]:
function print_dfs_codes(dfs)
    for i in 1:size(dfs)[1]
        x = dfs[i, :]
        print(string(x.h,x.f,x.l,x.p,x.r,x.s))
        print(",")
    end
end



print_dfs_codes (generic function with 1 method)

In [14]:
function rule_search1(dfs,maxtol=3)
    stats = [:h, :f, :l, :p, :r, :s]
    flag = true
    tol = 0
    while flag        
        for a in 1:6
            var = dfs[:, stats[a]]
            for i in 1:9
                filt1 = [false for ii in 1:size(dfs)[1]]
                signs = [">=", "==", "<="]
                if i == 1
                    signs = [">=", "=="]
                end
                if i == 9
                    signs = ["==", "<="]
                end
                for sign in signs
                    if sign == ">="
                        filt1 = var .>= i
                    elseif sign == "=="
                        filt1 = var .== i
                    else
                        filt1 = var .<= i
                    end
                    hits = filt1
                    if sum(hits) > 0
                        if maximum(dfs.c2[hits]) <= MXS
                            n_miss = sum(dfs.c2 .<= MXS) - sum(hits)
                            if n_miss <= tol
                                flag = false
                                print(string("Possible rule: ",stats[a],sign,i," with "))
                                println(string(n_miss," misses."))
                                miss_filt = (dfs.c2 .<= MXS) .&& .!(hits)
                                if n_miss > 0
                                    print_dfs_codes(dfs[miss_filt, :])
                                    println()
                                end
                            end
                        end
                    end
                end
            end
        end
        if flag
            tol += 1
        else
            return true
        end
        if tol > maxtol
            flag = false
        end
    end
    return false
end    


rule_search1 (generic function with 2 methods)

In [15]:
function rule_search2(dfs,maxtol=3)
    stats = [:h, :f, :l, :p, :r, :s]
    flag = true
    tol = 0
    while flag        
        for a in 1:5
            var1 = dfs[:, stats[a]]
            for i in 1:9
                filt1 = [false for ii in 1:size(dfs)[1]]
                signs1 = [">=", "==", "<="]
                if i == 1
                    signs1 = [">=", "=="]
                end
                if i == 9
                    signs1 = ["==", "<="]
                end
                for sign1 in signs1
                    if sign1 == ">="
                        filt1 = var1 .>= i
                    elseif sign1 == "=="
                        filt1 = var1 .== i
                    else
                        filt1 = var1 .<= i
                    end
                    for b in (a+1):6
                        var2 = dfs[:, stats[b]]
                        for j in 1:9
                            filt2 = [false for ii in 1:size(dfs)[1]]
                            signs2 = [">=", "==", "<="]
                            if j == 1
                                signs2 = [">=", "=="]
                            end
                            if j == 9
                                signs2 = ["==", "<="]
                            end
                            for sign2 in signs2
                                if sign2 == ">="
                                    filt2 = var2 .>= j
                                elseif sign2 == "=="
                                    filt2 = var2 .== j
                                else
                                    filt2 = var2 .<= j
                                end
                                for conj in ["and", "or"]
                                    hits = [false for ii in 1:size(dfs)[1]]
                                    if conj == "and"
                                        hits = filt1 .&& filt2
                                    else
                                        hits = filt1 .|| filt2
                                    end
                                    if sum(hits) > 0
                                        if maximum(dfs.c2[hits]) <= MXS
                                            n_miss = sum(dfs.c2 .<= MXS) - sum(hits)
                                            if n_miss <= tol
                                                flag = false
                                                print(string("Possible rule: ",stats[a],sign1,i," ",conj," "))
                                                print(string(stats[b],sign2,j," with "))
                                                println(string(n_miss," misses."))
                                                miss_filt = (dfs.c2 .<= MXS) .&& .!(hits)
                                                if n_miss > 0
                                                    print_dfs_codes(dfs[miss_filt, :])
                                                    println()
                                                end
                                            end
                                        end
                                    end
                                end
                            end
                        end
                    end
                end
            end
        end
        if flag
            tol += 1
        else
            return true
        end
        if tol > maxtol
            flag = false
        end
    end
    return false
end    

rule_search2 (generic function with 2 methods)

In [17]:
diff = [1,0,1,1,0,0];

In [18]:
final_vs = base_vs .+ transpose(diff)
final_vs[final_vs .> MXV] .= MXV;
cs = map(x -> scores[CartesianIndex(x...)], eachrow(final_vs))
sum(cs .<= MXS .&& cs .>= MXS - 5), maximum(cs)

(6, 127)

```
```

In [27]:
v1 = :l
v2 = :h
v3 = :f

df = spuds_to_df(bases)
df[:, :c2] = cs
for i in 1:9
    filt0 = df[:, v1] .== i 
    n0 = sum(df.c2[filt0] .<= MXS)
    if n0 > 0 && maximum(df.c2[filt0]) .<= MXS
        println()
        println(string(v1, "=", i, " with ",n0," eligible bases."))
        println("--no additional restrictions needed--")
    elseif n0 > 0
        for j in 1:9
            filt = df[:, v1] .== i .&& df[:, v2] .== j
            n = sum(df.c2[filt] .<= MXS)
            if n > 0
                println()
                println(string(v1, "=", i, " ", v2, "=", j, " with ",n," eligible bases."))
                dfs = df[filt,:]
                if maximum(dfs.c2) <= MXS
                    println("--no additional restrictions needed--")
                else
                    res = rule_search1(dfs)
                    if !res
                        res = rule_search2(dfs)
                        if !res
                            # search 3rd stat
                            for k in 1:9
                                filt2 = df[:, v1] .== i .&& df[:, v2] .== j .&& df[:, v3] .== k
                                n2 = sum(df.c2[filt2] .<= MXS)
                                if n2 > 0
                                    println(string(" ",v1, "=", i, " ", v2, "=", j, " ", v3, "=", k, " with ",n2," eligible bases."))
                                    dfs2 = df[filt2,:]
                                    if maximum(dfs2.c2) <= MXS
                                        println("--no additional restrictions needed--")
                                    else
                                        res = rule_search1(dfs2)
                                        if !res
                                            res = rule_search2(dfs2)
                                        end
                                    end
                                end
                            end
                        end
                    end
                end
            end
        end
    end
end


l=1 with 3510 eligible bases.
--no additional restrictions needed--

l=2 with 2826 eligible bases.
--no additional restrictions needed--

l=3 with 2030 eligible bases.
--no additional restrictions needed--

l=4 with 1290 eligible bases.
--no additional restrictions needed--

l=5 h=1 with 216 eligible bases.
--no additional restrictions needed--

l=5 h=2 with 165 eligible bases.
--no additional restrictions needed--

l=5 h=3 with 120 eligible bases.
--no additional restrictions needed--

l=5 h=4 with 84 eligible bases.
--no additional restrictions needed--

l=5 h=5 with 56 eligible bases.
--no additional restrictions needed--

l=5 h=6 with 35 eligible bases.
--no additional restrictions needed--

l=5 h=7 with 20 eligible bases.
--no additional restrictions needed--

l=6 h=1 with 120 eligible bases.
--no additional restrictions needed--

l=6 h=2 with 84 eligible bases.
--no additional restrictions needed--

l=6 h=3 with 56 eligible bases.
--no additional restrictions needed--

l=6 h=4 w

In [ ]:
dfs = df[df.s .== 3 .&& df.p .== 1,:]
println(dfs[sortperm(dfs.l),:])

In [31]:
println(df[sortperm(-df.c2)[1:50],:])

50×10 DataFrame
 Row │ name    h      f      l      p      r      s      a1     cost   c2    
     │ String  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Int64 
─────┼───────────────────────────────────────────────────────────────────────
   1 │ b21         8      1      6      1      1      1    999      0    127
   2 │ b356        7      1      6      1      2      1    999      0    112
   3 │ b2146       7      1      6      1      1      2    999      0    112
   4 │ b66         7      1      6      2      1      1    999      0    111
   5 │ b22         7      2      6      1      1      1    999      0    110
   6 │ b29         6      1      7      1      1      1    999      0    109
   7 │ b4026       8      1      5      1      1      3    999      0    106
   8 │ b2182       8      1      5      2      1      2    999      0    105
   9 │ b2456       8      1      5      1      2      2    999      0    105
  10 │ b392        8      1      5      2      2      1  